# BRFSS cleaning example

The BRFSS is a fairly large and messy dataset. To put it into a useable format, you have to do a bit of cleaning and massaging. This Notebook demonstrates cleaning the data bit by bit for 2016. Scripts for every year are in the `cleaning_code` folder. I had to do a separate script for each year because the codebooks change slightly nearly every year, and it's just enough to make it a real headache.

Pandas is a terrific tool for data cleaning and manipulation, and that's what I use here.


The `.XPT` file downloaded (from https://www.cdc.gov/brfss/annual_data/annual_2016.html -> https://www.cdc.gov/brfss/annual_data/2016/files/LLCP2016XPT.zip) was transformed into a `.csv` using the `foreign` package in `R`.

## Getting started
Let's start by importing our libraries. We'll be using Pandas and Numpy.

In [1]:
import numpy as np
import pandas as pd

Next, we'll set our year variable (2016), which made it easy to change when I had to copy scripts for each year.

In [2]:
year = '2016'

## Reading in the data

The original BRFSS data is available from the CDC in a couple of arcane formats (.XPT and .ASCII). I had to use R to get it into a nice CSV format. You can use my `sas2csv.R` script to make the translation if you like. I've done it for you and have stored the data on my Amazon server [link to raw csv data](https://www.amazon.com/clouddrive/share/HAfuNnNSbFqKmdyuodrVAQMpgcyqoFACuBoKWIqoWeG?ref_=cd_ph_share_link_copy).  This isn't available on GitHub because the file is too big. Also, 

Let's use Pandas to read in the `.csv` of the raw data for 2016. Since the file is so big, it might take some time to read, and you might even get a warning. We didn't specify UTF-8 encoding in the R script for creating the `.csv`'s, so we need to specify the encoding, which somehow ends up being the interesting choice of "cp1252." That little tidbit right there was a couple hours worth of Googling.

In [3]:
df = pd.read_csv('brfss' + year + '.csv', encoding='cp1252')

/opt/packages/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (131) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Let's take a quick look at what we have here. As you can see, this is a rather large data set.

In [4]:
df

,Unnamed: 0,X_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,X_PSU,...,X_MAM5021,X_RFPAP33,X_RFPSA21,X_RFBLDS3,X_COL10YR,X_HFOB3YR,X_FS5YR,X_FOBTFS,X_CRCREC,X_AIDTST3
0,1,1,1,1072016,1,7,2016,1100,2016000001,2016000001,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2,1,1,1112016,1,11,2016,1100,2016000002,2016000002,...,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,2.0
2,3,1,1,1062016,1,6,2016,1100,2016000003,2016000003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,4,1,1,1082016,1,8,2016,1100,2016000004,2016000004,...,NaN,NaN,1.0,2.0,1.0,2.0,NaN,2.0,1.0,9.0
4,5,1,1,1052016,1,5,2016,1100,2016000005,2016000005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486298,486299,78,12,12312016,12,31,2016,1200,2016001262,2016001262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486299,486300,78,12,12192016,12,19,2016,1100,2016001263,2016001263,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
486300,486301,78,12,12092016,12,9,2016,1100,2016001264,2016001264,...,2.0,1.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0
486301,486302,78,12,12312016,12,31,2016,1200,2016001265,2016001265,...,2.0,1.0,NaN,2.0,NaN,2.0,NaN,2.0,NaN,NaN


In [5]:
print("Number of rows:", len(df))
print("Number of columns (variables in the survey):", len(df.columns))
print(df.columns)

Number of rows: 486303
Number of columns (variables in the survey): 276
Index(['Unnamed: 0', 'X_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR',
       'DISPCODE', 'SEQNO', 'X_PSU',
       ...
       'X_MAM5021', 'X_RFPAP33', 'X_RFPSA21', 'X_RFBLDS3', 'X_COL10YR',
       'X_HFOB3YR', 'X_FS5YR', 'X_FOBTFS', 'X_CRCREC', 'X_AIDTST3'],
      dtype='object', length=276)


## Collecting our variables of interest

Great, so we have a lot of rows and 280 columns. We're just trying to find some basic demographic information. If we look through the codebook (https://www.cdc.gov/brfss/annual_data/2016/pdf/codebook16_llcp.pdf), we can find those variable names. Note that an underscore in the codebook is the same as `X_` in the data, and that variables are all caps in the codebook but lowercase in the data.

- **Income:** INCOME2
- **Race:** X_RACE
- **State:** X_STATE
- **Age:** X_AGEG5YR
- **Sex:** SEX
- **BMI:** X_BMI5CAT
- **Smoker:** X_SMOKER3
- **Heavy Drinker:** X_RFDRHV5

We can use Pandas to easily grab the columns we want. We're going to grab those columns as Pandas Series, clean them up, and then piece everything back together as a master DataFrame.

In [6]:
income = df['INCOME2']
race = df['X_RACE']
state = df['X_STATE']
age = df['X_AGEG5YR']
sex = df['SEX']
bmi = df['X_BMI5CAT']
smoker = df['X_SMOKER3']
diabetes = df["DIABETE3"]
heavy_drinker = df["X_RFDRHV5"]
# If you want to add more variables to your dataset add them here!

If we look at one of these variables, we can get a peek at the way the survey data is coded.

In [7]:
income

0          5.0
1          7.0
2          7.0
3          7.0
4         77.0
          ... 
486298     1.0
486299    99.0
486300     4.0
486301     4.0
486302     1.0
Name: INCOME2, Length: 486303, dtype: float64

##Cleaning up a column: Income

We'll start with cleaning up the income data. Looking at the codebook, the codes are:

* 1: <\$10k
* 2: \$10k - \$15k
* 3: \$15k - \$20k
* 4: \$20k - \$25k
* 5: \$25k - \$35k
* 6: \$35k - \$50k
* 7: \$50k - \$75k
* 8: \>\$75k
* 77: Unknown
* 99: Refused

We're going to group 77 and 99 together as `np.nan`, and we're going to group 7 and 8 together as simply \>\$50k (in earlier BRFSS years, it stops at />\$50K, rather than including the \$50k-\$75k and \>\$75k groups)

Pandas makes it easy to use a dictionary for replacement, so we'll put these codes into a dictionary.

In [8]:
income_replace = {1:'<10k', 2:'10k-15k', 3:'15k-20k', 4:'20k-25k', 5:'25k-35k', 6:'35k-50k', 7:'>50k', 8:'>50k', 77:np.nan, 99:np.nan}

## Cleaning the other columns

Next, we'll clean the race column. If you are working with historical data, this is the trickiest column. The name of the variable, the meanings of the codes, and the groups represented change frequently over the years, and sometimes in subtle ways. Being able to use historical data is why my dictionary ends up grouping some things together that the 2016 dataset breaks out. See the codebook for more details.

In [9]:
race_replace = {1:'white', 2:'black', 3:'native american', 4:'asian/non-hispanic', 5:'native hawaiian', 6:'other/non-hispanic', 7:'multiracial', 8:'hispanic', 9:'refused/unknown'}

We'll skip the state column for now. It's represented as a FIPS statecode, and you can find more info on Wikipedia: [FIPS codes](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code).

Let's get the age and sex columns mapped.

In [10]:
age_replace = {1:'18-24', 2:'25-29', 3:'30-34', 4:'35-39', 5:'40-44', 6:'45-49', 7:'50-54', 8:'55-59', 9:'60-64', 10:'65-69', 11:'70-74', 12:'75-79', 13:'80+', 14:np.nan}
sex_replace = {1:'male', 2:'female'}

Now for the states...

In [11]:
state_replace = {1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming', 66: 'Guam', 72: 'Puerto Rico', 78: 'Virgin Islands'}

Finally we'll do the bmi, smoker, diabetes, and heavy_drinker columns

In [12]:
bmi_replace = {1: 'underweight', 2: 'normal', 3: 'overweight', 4: 'obese'}
smoker_replace = {1: 'current smoker', 2: 'current smoker', 3: 'former smoker', 4: 'never smoked', 9:'refused/unknown'}
diabetes_replace = {1: 'yes', 2: 'yes', 3: 'no', 4:'no', 7:np.nan, 9:np.nan}
heavy_drinker_replace = {1: 'no', 2:'yes', 9:np.nan}

## Using Pandas to do the replacement
Since we've got our dictionaries of codes mapped to values, we can let Pandas do the heavy lifting of cleaning up our columns.

In [13]:
income = income.replace(income_replace)
race = race.replace(race_replace)
age = age.replace(age_replace)
sex = sex.replace(sex_replace)
state = state.replace(state_replace)
bmi = bmi.replace(bmi_replace)
smoker = smoker.replace(smoker_replace)
diabetes = diabetes.replace(diabetes_replace)
heavy_drinker = heavy_drinker.replace(heavy_drinker_replace)

Let's check out one of our cleaned up columns. Now it is beautiful and easy-to-use:

In [14]:
diabetes

0          no
1          no
2          no
3         yes
4          no
         ... 
486298     no
486299     no
486300     no
486301     no
486302     no
Name: DIABETE3, Length: 486303, dtype: object

## Saving our clean data

We're almost there! Now, let's take all of our cleaned up columns and put them back together as a single DataFrame and name the columns something easy to understand. We'll save that out to a `.csv`.

In [15]:
brfss_out = pd.concat([income, race, state, age, sex, bmi, smoker, diabetes, heavy_drinker], axis=1)
brfss_out.columns = ['income', 'race', 'state', 'age', 'sex', 'bmi', 'smoker', 'diabetes', 'heavy_drinker']
brfss_out.to_csv('brfss' + year + 'clean.csv')

And now, at long last, we are finished. We have a cleaned up set of demographic data in `brfss2016clean.csv`. Now, let's try and load it into a new DataFrame.

In [16]:
clean_df = pd.read_csv('brfss' + year + 'clean.csv', encoding='cp1252')

Let's make sure it looks right!

In [17]:
clean_df

,Unnamed: 0,income,race,state,age,sex,bmi,smoker,diabetes,heavy_drinker
0,0,25k-35k,white,Alabama,40-44,male,normal,never smoked,no,no
1,1,>50k,white,Alabama,55-59,female,overweight,never smoked,no,no
2,2,>50k,white,Alabama,80+,female,normal,current smoker,no,no
3,3,>50k,white,Alabama,70-74,male,overweight,former smoker,yes,no
4,4,NaN,white,Alabama,18-24,male,normal,never smoked,no,no
...,...,...,...,...,...,...,...,...,...,...
486298,486298,<10k,hispanic,Virgin Islands,30-34,female,overweight,refused/unknown,no,NaN
486299,486299,NaN,black,Virgin Islands,40-44,male,normal,never smoked,no,NaN
486300,486300,20k-25k,black,Virgin Islands,60-64,female,obese,former smoker,no,no
486301,486301,20k-25k,black,Virgin Islands,50-54,female,obese,never smoked,no,no


In [18]:
clean_df.query("state=='Guam'")

,Unnamed: 0,income,race,state,age,sex,bmi,smoker,diabetes,heavy_drinker
477665,477665,>50k,asian/non-hispanic,Guam,65-69,male,normal,current smoker,no,no
477666,477666,NaN,asian/non-hispanic,Guam,50-54,female,overweight,never smoked,no,NaN
477667,477667,10k-15k,hispanic,Guam,55-59,female,overweight,former smoker,no,no
477668,477668,<10k,native hawaiian,Guam,18-24,male,normal,never smoked,no,no
477669,477669,>50k,native hawaiian,Guam,35-39,female,obese,former smoker,no,no
...,...,...,...,...,...,...,...,...,...,...
479238,479238,NaN,native hawaiian,Guam,18-24,female,obese,never smoked,no,no
479239,479239,>50k,white,Guam,30-34,male,overweight,former smoker,no,no
479240,479240,>50k,native hawaiian,Guam,45-49,female,normal,never smoked,no,no
479241,479241,<10k,native hawaiian,Guam,30-34,male,normal,former smoker,no,yes


In [19]:
clean_df.query("state=='Guam' and \
                diabetes=='yes' and \
                (smoker=='current smoker' or smoker=='former smoker')")

,Unnamed: 0,income,race,state,age,sex,bmi,smoker,diabetes,heavy_drinker
477676,477676,<10k,hispanic,Guam,35-39,female,obese,current smoker,yes,no
477751,477751,10k-15k,native hawaiian,Guam,40-44,male,overweight,former smoker,yes,no
477760,477760,10k-15k,native hawaiian,Guam,60-64,male,obese,current smoker,yes,no
477764,477764,10k-15k,native hawaiian,Guam,50-54,male,overweight,current smoker,yes,no
477775,477775,10k-15k,asian/non-hispanic,Guam,75-79,female,normal,former smoker,yes,no
...,...,...,...,...,...,...,...,...,...,...
479178,479178,25k-35k,asian/non-hispanic,Guam,35-39,male,overweight,current smoker,yes,no
479192,479192,NaN,hispanic,Guam,65-69,female,overweight,former smoker,yes,no
479205,479205,>50k,asian/non-hispanic,Guam,40-44,female,overweight,current smoker,yes,no
479216,479216,10k-15k,native hawaiian,Guam,55-59,male,overweight,current smoker,yes,no


In [20]:
clean_df.query("state=='Guam' and \
                heavy_drinker=='yes'")

,Unnamed: 0,income,race,state,age,sex,bmi,smoker,diabetes,heavy_drinker
477684,477684,<10k,native hawaiian,Guam,50-54,male,obese,never smoked,no,yes
477689,477689,25k-35k,hispanic,Guam,45-49,male,overweight,current smoker,no,yes
477692,477692,20k-25k,native hawaiian,Guam,50-54,male,obese,never smoked,no,yes
477694,477694,>50k,hispanic,Guam,55-59,male,overweight,current smoker,no,yes
477728,477728,>50k,white,Guam,35-39,male,obese,former smoker,no,yes
...,...,...,...,...,...,...,...,...,...,...
479224,479224,>50k,native hawaiian,Guam,40-44,male,overweight,current smoker,no,yes
479230,479230,>50k,native hawaiian,Guam,45-49,male,overweight,former smoker,no,yes
479236,479236,35k-50k,hispanic,Guam,18-24,female,normal,never smoked,no,yes
479241,479241,<10k,native hawaiian,Guam,30-34,male,normal,former smoker,no,yes
